# Cleaning data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("archive/steam_game_review.csv")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df

,appid,success,query_summary,reviews,cursor
0,1020470,1,"{'num_reviews': 2, 'review_score': 6, 'review_...","[{'recommendationid': '154413393', 'author': {...",AoIIPwYYaneoldUE
1,1018050,1,"{'num_reviews': 0, 'review_score': 0, 'review_...",[],*
2,1018060,1,"{'num_reviews': 0, 'review_score': 0, 'review_...",[],*
3,1018080,1,"{'num_reviews': 3, 'review_score': 0, 'review_...","[{'recommendationid': '55936147', 'author': {'...",AoIIAAAAAHTXjPUB
4,1018090,1,"{'num_reviews': 7, 'review_score': 0, 'review_...","[{'recommendationid': '49140086', 'author': {'...",AoIIAAAAAHOi3rYD
...,...,...,...,...,...
94021,1045590,1,"{'num_reviews': 0, 'review_score': 0, 'review_...",[],*
94022,1045600,1,"{'num_reviews': 0, 'review_score': 0, 'review_...",[],*
94023,1045610,1,"{'num_reviews': 2, 'review_score': 5, 'review_...","[{'recommendationid': '152497829', 'author': {...",AoIIPwAAAHqwhscE
94024,1045630,1,"{'num_reviews': 0, 'review_score': 0, 'review_...",[],*


In [4]:
df["success"].unique().shape, df["cursor"].unique().shape

((1,), (45370,))

In [5]:
df.drop(columns=["success", "cursor"], inplace=True)

## Convert from JSON (text) to columns

___

In [6]:
def simple_string_to_json(x, returning={}):
    if pd.isna(x):
        return returning
    else:
        return json.loads(x.replace("\'", "\""))

In [7]:
df["query_summary"] = df["query_summary"].apply(simple_string_to_json)
df["query_summary"]

0        {'num_reviews': 2, 'review_score': 6, 'review_...
1        {'num_reviews': 0, 'review_score': 0, 'review_...
2        {'num_reviews': 0, 'review_score': 0, 'review_...
3        {'num_reviews': 3, 'review_score': 0, 'review_...
4        {'num_reviews': 7, 'review_score': 0, 'review_...
                               ...                        
94021    {'num_reviews': 0, 'review_score': 0, 'review_...
94022    {'num_reviews': 0, 'review_score': 0, 'review_...
94023    {'num_reviews': 2, 'review_score': 5, 'review_...
94024    {'num_reviews': 0, 'review_score': 0, 'review_...
94025    {'num_reviews': 20, 'review_score': 8, 'review...
Name: query_summary, Length: 94026, dtype: object

In [12]:
def string_with_apostrophe_brute_force_to_json(x, returning={}):
	temp = ""
	try:
		temp = x.replace("\"", "'")			\
				.replace("'}", "\"}").replace("\\xa0", "")					\
				.replace("\\", "")											\
				.replace("'recommendationid': '",         "\"recommendationid\": \"")			\
				.replace("', 'author': {'steamid': '",    "\", \"author\": {\"steamid\": \"")	\
				.replace("', 'num_games_owned'",          "\", \"num_games_owned\"")			\
				.replace("'num_reviews'",                 "\"num_reviews\"")					\
				.replace("'playtime_forever'",            "\"playtime_forever\"")				\
				.replace("'playtime_at_review'",          "\"playtime_at_review\"")				\
				.replace("'playtime_last_two_weeks'",     "\"playtime_last_two_weeks\"")		\
				.replace("'last_played'",                 "\"last_played\"")					\
				.replace("'language': '",                 "\"language\": \"")					\
				.replace("'timestamp_updated'",           "\"timestamp_updated\"")				\
				.replace("'voted_up'",                    "\"voted_up\"")						\
				.replace("'votes_up'",                    "\"votes_up\"")						\
				.replace("'votes_funny'",                 "\"votes_funny\"")					\
				.replace("'weighted_vote_score'",         "\"weighted_vote_score\"")			\
				.replace("'comment_count'",               "\"comment_count\"")					\
				.replace("'steam_purchase'",              "\"steam_purchase\"")					\
				.replace("'received_for_free'",           "\"received_for_free\"")	 			\
				.replace("'written_during_early_access'", "\"written_during_early_access\"")	\
				.replace("'hidden_in_steam_china'",       "\"hidden_in_steam_china\"")			\
				.replace("'steam_china_location': '",     "\"steam_china_location\": \"")		\
				.replace("True", "true").replace("False", "false")

		temp = re.sub(r": '(\d+.?\d*)'", r': "\1"', temp)
		temp = re.sub(r"', 'review': ['\"]", "\", \"review\": \"", temp)
		temp = re.sub(r"['\"], 'timestamp_created'", "\", \"timestamp_created\"", temp)

		temp = temp.replace("\\'", "'")
		return json.loads(temp)		# Why can't load UNICODE properly

	except Exception as e:
		print(x)
		print(temp)
		print(e, '\n')
		return returning

In [15]:
count = 0

def string_with_apostrophe_to_json(x, returning={}):
	global count
	count += 1
	if count % 10000 == 0:
		print(count)
	
	if pd.isna(x):
		return returning
	else:
		temp = ""
		try:
			temp = x.replace("\"", "'").replace("\\'", "'")
			temp = re.sub(r"([\[\{\n]|[:,] )\'", r'\1"', temp)			# Left quotation mark: [ '[' '{' '\n' ': ' ]
			temp = re.sub(r"\'([\]\}\n:]|, \"|: \")", r'"\1', temp)		# Right quotation mark
			temp = temp.replace("True", "true").replace("False", "false")
			return json.loads(temp)
		except Exception as e:
			return string_with_apostrophe_brute_force_to_json(x, returning)


# string_with_apostrophe_to_json(df.loc[100, "reviews"])
# t = df["reviews"].apply(lambda x: string_with_apostrophe_to_json(x, []))
df["reviews"] = df["reviews"].apply(lambda x: string_with_apostrophe_to_json(x, []))

In [20]:
#df = df[df["reviews"].apply(lambda x: len(x) > 0)]

In [8]:
col_list = df.loc[0, "query_summary"].keys()
col_list

dict_keys(['num_reviews', 'review_score', 'review_score_desc', 'total_positive', 'total_negative', 'total_reviews'])

In [9]:
for col in col_list:
	df[col] = df["query_summary"].apply(lambda x: x[col])
df.drop(columns=["query_summary"], inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94026 entries, 0 to 94025
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   appid              94026 non-null  int64 
 1   reviews            94026 non-null  object
 2   num_reviews        94026 non-null  int64 
 3   review_score       94026 non-null  int64 
 4   review_score_desc  94026 non-null  object
 5   total_positive     94026 non-null  int64 
 6   total_negative     94026 non-null  int64 
 7   total_reviews      94026 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 5.7+ MB


Fortunately, there is no empty value.

## Extract reviews

___

In [26]:
columns = df.loc[1029, "reviews"][0]["author"].keys()
columns

dict_keys(['steamid', 'num_games_owned', 'num_reviews', 'playtime_forever', 'playtime_last_two_weeks', 'playtime_at_review', 'last_played'])

In [27]:
pd.DataFrame.from_records(df.loc[0, "reviews"])

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,154413393,"{'steamid': '76561198971613275', 'num_games_ow...",english,Evoland 3 wen?,1703552228,1703552228,True,2,1,0.528369247913360596,0,True,False,False,True,
1,156543623,"{'steamid': '76561198267303077', 'num_games_ow...",english,I'll be leaving the comment in English for bet...,1705924660,1705924660,True,1,0,0.523809552192687988,0,True,False,False,True,


In [44]:
# DO NOT RUN THIS SINCE IT WILL TALK 20 MINUTES TO FULLY COMPILE
df_review_clean = pd.DataFrame()

tiny_author = None

for i in range(df.shape[0]):
	i += 1
	if i % 3000 == 0:
		print(i)

	try:
		tiny_author = pd.DataFrame.from_records(df.loc[i, "reviews"])
		tiny_author["appid"] = df.loc[i, "appid"]
		if tiny_author.empty:
			continue
		for col in columns:
			tiny_author[col] = tiny_author["author"].apply(lambda x: x.get(col, None))
		tiny_author.drop(columns=["author"], inplace=True)
		df_review_clean = pd.concat([df_review_clean, tiny_author], axis=0)

	except Exception as e:
		print(i)
		print(tiny_author)
		print(e)


df_review_clean

3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
94026
   recommendationid language  \
0         108778272  english   
1          50377239  english   
2          65225574  english   
3          70609466  english   
4          49825224  english   
5          51192716  english   
6          49983377  english   
7          77041914  english   
8          76060023  english   
9          69073364  english   
10         64981685  english   
11         50952534  english   
12         52642850  english   
13         50444814  english   
14         49796613  english   
15         49896966  english   
16         50474577  english   
17         86551407  english   
18        130122557  english   
19         61947767  english   

                                               review  timestamp_created  \
0                               very nice, enjoyed it         

,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,...,appid,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,timestamp_dev_responded,developer_response
0,55936147,english,A fun and quirky stealth-based problem solving...,1572489221,1572489221,True,2,0,0.554140150547027588,0,...,1018080,76561198051821837,0,12,41,0,41,1572483146,NaN,NaN
1,55989797,english,"Loved the art style, and the game ran very smo...",1572523060,1572523060,True,1,0,0.523809552192687988,0,...,1018080,76561197993790846,657,8,17,0,15,1572773870,NaN,NaN
2,64251252,english,the game crashed four times for one hour.... i...,1582906915,1582907209,False,0,0,0,0,...,1018080,76561198095855343,1254,24,76,0,76,1582906567,NaN,NaN
0,49140086,english,While I cannot recommend this Unity asset reli...,1550764336,1550764336,False,16,0,0.624971091747283936,0,...,1018090,76561198053422627,2384,1225,56,0,56,1550955553,NaN,NaN
1,49137406,english,Is extremely unoptimized and has laggy framera...,1550754621,1550754621,False,11,0,0.527824163436889648,0,...,1018090,76561198019816374,1351,1674,10,0,10,1550748455,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,49896966,english,Simple and Fun,1554138916,1554138916,True,1,0,0.476877689361572266,0,...,1045650,76561198370598894,0,4,347,0,32,1645556008,NaN,NaN
16,50474577,english,its a bad game but its not gay so that good,1556982400,1556982400,True,1,4,0.466409146785736084,0,...,1045650,76561198200188689,99,38,25,0,22,1605293404,NaN,NaN
17,86551407,english,[h1] Review: [/h1]\nI bought it because 0.39€ ...,1613229614,1613229667,False,0,0,0.454661726951599121,0,...,1045650,76561198418292141,770,13,15,0,15,1613228000,NaN,NaN
18,130122557,english,"the car drifts for no reason, making the game ...",1672852164,1672852164,False,0,0,0.454545468091964722,0,...,1045650,76561199137672724,0,12,53,0,35,1672853552,NaN,NaN


In [8]:
# df_review_clean.to_csv("archive/cleaned_reviews_v1.csv")
df_review_clean = pd.read_csv("archive/cleaned_reviews_v1.csv")
df_review_clean = df_review_clean.loc[:, ~df_review_clean.columns.str.contains("^Unnamed")]

It's safe to drop the feature with huge storage memory

In [11]:
df.drop(columns=["reviews"], inplace=True)

In [13]:
df.head()

,appid,num_reviews,review_score,review_score_desc,total_positive,total_negative,total_reviews
0,1020470,2,6,Mostly Positive,360,106,466
1,1018050,0,0,No user reviews,0,0,0
2,1018060,0,0,No user reviews,0,0,0
3,1018080,3,0,3 user reviews,2,1,3
4,1018090,7,0,7 user reviews,1,6,7


In [14]:
df.to_csv("archive/cleaned_bridge.csv")

In [9]:
df_review_clean.reset_index(drop=True, inplace=True)
df_review_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299341 entries, 0 to 299340
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   recommendationid             299341 non-null  int64  
 1   language                     299341 non-null  object 
 2   review                       299220 non-null  object 
 3   timestamp_created            299341 non-null  int64  
 4   timestamp_updated            299341 non-null  int64  
 5   voted_up                     299341 non-null  bool   
 6   votes_up                     299341 non-null  int64  
 7   votes_funny                  299341 non-null  int64  
 8   weighted_vote_score          299341 non-null  float64
 9   comment_count                299341 non-null  int64  
 10  steam_purchase               299341 non-null  bool   
 11  received_for_free            299341 non-null  bool   
 12  written_during_early_access  299341 non-null  bool   
 13 

## Null treatment

___

In [10]:
# Lots of empty values
df_review_clean.drop(columns=["timestamp_dev_responded", "developer_response", "steam_china_location"], inplace=True)

Before filling null to `playtime_at_review`, let's see the data distribution

In [6]:
from collections import Counter
Counter(df_review_clean["playtime_at_review"].sort_values())

Counter({6.0: 3002,
         5.0: 2821,
         7.0: 2608,
         10.0: 2400,
         8.0: 2395,
         9.0: 2361,
         11.0: 2276,
         13.0: 2129,
         30.0: 2102,
         12.0: 2051,
         15.0: 1967,
         14.0: 1932,
         16.0: 1833,
         17.0: 1690,
         18.0: 1689,
         19.0: 1680,
         20.0: 1658,
         21.0: 1579,
         22.0: 1567,
         23.0: 1458,
         24.0: 1417,
         25.0: 1417,
         26.0: 1413,
         27.0: 1398,
         28.0: 1292,
         33.0: 1284,
         60.0: 1283,
         29.0: 1268,
         31.0: 1241,
         32.0: 1187,
         35.0: 1151,
         34.0: 1111,
         36.0: 1102,
         37.0: 1099,
         38.0: 1039,
         39.0: 1030,
         40.0: 1003,
         44.0: 989,
         41.0: 984,
         42.0: 958,
         45.0: 944,
         46.0: 939,
         43.0: 919,
         48.0: 905,
         47.0: 904,
         51.0: 841,
         61.0: 836,
         50.0: 835,
        

The most frequent value is `6`. However, the number of empty values are 67780. This means filling null with `6` doesn't make sense.

Let's try filling null with a simple machine learning model. Since it's a quick filling, it's better to use a high accurate model.

In [48]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [49]:
feature_table = df_review_clean[['voted_up', 'votes_up', 'votes_funny',
						 'comment_count', 'num_games_owned', 'num_reviews',
						 'playtime_forever', 'playtime_last_two_weeks', 'timestamp_created', 'playtime_at_review']]

train_feature_table = feature_table[feature_table["playtime_at_review"].notna()]
test_feature_table  = feature_table[feature_table["playtime_at_review"].isna()]

X_train, X_test, y_train, y_test = train_test_split(train_feature_table.drop(columns=["playtime_at_review"]), train_feature_table["playtime_at_review"])

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((172529, 9), (57510, 9), (172529,), (57510,))

In [55]:
r2_score_save = {}
model_save = {}

for i in range(2, 10):
	tree_model = DecisionTreeRegressor(max_depth=i)
	tree_model.fit(X_train, y_train)
	y_pred = tree_model.predict(X_test)
	r2_score_save[i] = r2_score(y_test, y_pred)
	model_save[i] = tree_model

r2_score_save

{2: 0.32008022017552407,
 3: 0.11618521832603612,
 4: 0.13341820563534168,
 5: 0.8321167670047849,
 6: -0.07676525987167837,
 7: 0.33120855669347005,
 8: 0.8289659663942521,
 9: 0.5213046576700924}

Since decision tree can encounter overfitting, we need to select the case which has high R2 score at prediction.

In [58]:
best_depth = max(r2_score_save, key=r2_score_save.get)
test_feature_table["playtime_at_review"] = model_save[best_depth].predict(test_feature_table.drop(columns=["playtime_at_review"]))

And fill the empty values

In [62]:
for i in test_feature_table.index:
	df_review_clean.loc[i, "playtime_at_review"] = test_feature_table.loc[i, "playtime_at_review"]

df_review_clean["playtime_at_review"].isna().sum()

0

Fill the empty review with empty string

In [66]:
df_review_clean["review"].fillna("", inplace=True)

In [67]:
df_review_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299341 entries, 0 to 299340
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   recommendationid             299341 non-null  int64  
 1   language                     299341 non-null  object 
 2   review                       299341 non-null  object 
 3   timestamp_created            299341 non-null  int64  
 4   timestamp_updated            299341 non-null  int64  
 5   voted_up                     299341 non-null  bool   
 6   votes_up                     299341 non-null  int64  
 7   votes_funny                  299341 non-null  int64  
 8   weighted_vote_score          299341 non-null  float64
 9   comment_count                299341 non-null  int64  
 10  steam_purchase               299341 non-null  bool   
 11  received_for_free            299341 non-null  bool   
 12  written_during_early_access  299341 non-null  bool   
 13 

## Type rectification

___

In [69]:
# Drop unnecessary features
df_review_clean.drop(columns=["hidden_in_steam_china"], inplace=True)

In [70]:
# Weighted -> number
df_review_clean["weighted_vote_score"] = df_review_clean["weighted_vote_score"].astype(float)
# ID should be a string
df_review_clean["appid"] = df_review_clean["appid"].astype(str)

In [76]:
df_review_clean["timestamp_created"] = pd.to_datetime(df_review_clean["timestamp_created"], unit='s')
df_review_clean["timestamp_updated"] = pd.to_datetime(df_review_clean["timestamp_updated"], unit='s')
df_review_clean["last_played"] = pd.to_datetime(df_review_clean["last_played"], unit='s')
df_review_clean[["timestamp_created", "timestamp_updated", "last_played"]].head()

,timestamp_created,timestamp_updated,last_played
0,2019-10-31 02:33:41,2019-10-31 02:33:41,2019-10-31 00:52:26
1,2019-10-31 11:57:40,2019-10-31 11:57:40,2019-11-03 09:37:50
2,2020-02-28 16:21:55,2020-02-28 16:26:49,2020-02-28 16:16:07
3,2019-02-21 15:52:16,2019-02-21 15:52:16,2019-02-23 20:59:13
4,2019-02-21 13:10:21,2019-02-21 13:10:21,2019-02-21 11:27:35


In [23]:
# Drop this column as well since it only has one kind of value
print(df_review_clean["language"].unique())
df_review_clean.drop(columns=["language"], inplace=True)

['english']


## Finish

___

In [24]:
df_review_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299341 entries, 0 to 299340
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   recommendationid             299341 non-null  int64  
 1   review                       299220 non-null  object 
 2   timestamp_created            299341 non-null  object 
 3   timestamp_updated            299341 non-null  object 
 4   voted_up                     299341 non-null  bool   
 5   votes_up                     299341 non-null  int64  
 6   votes_funny                  299341 non-null  int64  
 7   weighted_vote_score          299341 non-null  float64
 8   comment_count                299341 non-null  int64  
 9   steam_purchase               299341 non-null  bool   
 10  received_for_free            299341 non-null  bool   
 11  written_during_early_access  299341 non-null  bool   
 12  appid                        299341 non-null  int64  
 13 

In [26]:
df_review_clean.to_csv("archive/cleaned_reviews_v2.csv", index=False)